In [1]:
import pandas as pd
import numpy as np
df_path = "./twitter_all.csv"
df = pd.read_csv(df_path, sep=',', encoding='utf-8') 

In [5]:
Template = '''{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}'''


In [2]:
df.head()

,tweet,full_name,brand
0,신사 골뱅이신사\n\n골뱅이 존맛 배터지게 먹고왔어용\n#정한아_마니머거여ㅎㅎ\n#...,골뱅이신사 신사,골뱅이신사
1,더보이즈 투어 폼 미춋다..\n예절샷 못 찍는 사람들의 만남\n 벽돌해피푸드 압구정...,벽돌해피푸드 압구정 / 디저트로이 압구정로데오 / 파스텔드나따 압구정 / 한스 압구...,벽돌해피푸드 / 디저트로이 / 파스텔드나따 / 한스 / 카츠바이콘반
2,코엑스 스시한판\n\n직원이 졸라 불친절해서 개빡쳤지만\n초밥 개맛있어서 화가 녹아...,스시한판 코엑스,스시한판
3,민현이가 좋아하는 압구정 로데오 포케 맛집\n보울룸 <Bowl Room>\n이라고합...,보울룸 압구정,보울룸
4,진짜 맛있게 배터지게 먹고 엄청 웃어서 소화 다 됐다.. 나나투어 직전에 벌떡 일어...,뱃고동 압구정,뱃고동


In [3]:
system_message = "You need to extract brand name from the korean tweeter message. Follow the examples."

messages = []
for i in range(len(df)):
    if (df['tweet'][i] is float('nan')) or (df['brand'][i] is float('nan')):
        continue
    prompt = df['tweet'][i]
    response = df['brand'][i]
    m = {"tweet_message": prompt,
         "brand_name": response}
    messages.append(m)


In [4]:
# create fine tune dataset
import json
finetune_dataset = []
for message in messages:
    user_message = message['tweet_message']
    assistant_message = message['brand_name']
    finetune_messages = [
        {"role": "system", "content": str(system_message)},
        {"role": "user", "content": str(user_message)},
        {"role": "assistant", "content": str(assistant_message)}
    ]
    finetune_dataset.append({"messages": finetune_messages})


In [10]:
with open("finetune_dataset_twitter_train.jsonl", "w",encoding="utf-8") as outfile:
    for item in finetune_dataset[:19000]:
        json_line = json.dumps(item, ensure_ascii=False)
        outfile.write(json_line + "\n")

with open("finetune_dataset_twitter_test.jsonl", "w",encoding="utf-8") as outfile:
    for item in finetune_dataset[19000:]:
        json_line = json.dumps(item, ensure_ascii=False)
        outfile.write(json_line + "\n")

In [11]:
# load json
with open('finetune_dataset_twitter_train.jsonl', 'r',encoding="utf-8") as f:
    train_data = f.readlines()
    print(train_data[0])


{"messages": [{"role": "system", "content": "You need to extract brand name from the korean tweeter message. Follow the examples."}, {"role": "user", "content": "신사 골뱅이신사\n\n골뱅이 존맛 배터지게 먹고왔어용\n#정한아_마니머거여ㅎㅎ\n#믿고먹어boo세요"}, {"role": "assistant", "content": "골뱅이신사"}]}



In [12]:
import dotenv
dotenv.load_dotenv()
from langchain.chains import create_extraction_chain
from langchain_openai import ChatOpenAI


# Schema
schema = {
    "properties": {
        "brand": {"type": "string"},
        "sentiment": {"type": "integer"},
        "geological_position": {"type": "string"},
    },
    "required": ["brand"],
}

# Input
inp = """신사 골뱅이신사

골뱅이 존맛 배터지게 먹고왔어용
#정한아_마니머거여ㅎㅎ
#믿고먹어boo세요

"""


# Run chain
llm = ChatOpenAI(temperature=0.1, model="gpt-4")
chain = create_extraction_chain(schema, llm, prompt = )
chain.run(inp)




[{'brand': '신사 골뱅이신사', 'sentiment': 1}]

In [15]:
for i in messages[20:41]:
    print({'tweet_message': i['tweet_message']})

{'tweet_message': '돌체테리아 압구정점\n\n더보이즈 없는 IST가서 사진 찍은 그날\nIST 주변 카페 들어가서 더보이즈 못 본 한을 풀었던 그날\n휘낭시에가 너무 맛있어서 순삭했던 그날\n@ju_yoo0115\n#ㄷㅂㅇㅈ\n#김선우_맛있는걸내가놔둘리가\n#손영재_콧구멍커지겠지\n#주연앟_맛있겠짛\n#지창민_도토리대신'}
{'tweet_message': '압구정로데오 대막 & 어글리퍼피 도산\n\n바질소바랑 후토마끼 맛있어요 !\n둘이서 2인세트 먹으니까 딱 좋았어요\n애플파이가 진짜 미쳤으니까 추천 !!\n딸기타르트도 맛있지만 애플파이가 진짜 입니다 ...\n\n#박지성_단1g도안줌\n#재혀니_맛있게머겅'}
{'tweet_message': '#박지성_단1g도안줌\n 코엑스, 아웃백\n\n친구들이 두시간 전에 미리 웨이팅 걸어둔 덕분에 맛있게 잘 먹었다 ( ¤̴̶̷̤́ ‧̫̮ ¤̴̶̷̤̀ ) ..'}
{'tweet_message': '마요네즈 신논현점\n\n#더보이즈_배고플땐언제든찾아와요\n#영훈아_빵빵하게_먹어\n#어이어이_이재현_여기다\n#ㄷㅂㅇㅈ\n#더비예절\n\n연말 케이팝 모임 '}
{'tweet_message': '#승연아_우즈야_여기야\n#우무이\n웍셔너리 압구정 (도산)\n\n벌써 3번째 방문….// 웨이팅 쫌 했지만 존맛…\n갈때마다 무조건 시키는 차오미엔이랑\n오렌지 치킨 여전히 너무 맛있음…\n몽골리안비프 누들 첨 시켜봤는데 \n실망시키지 않는 불맛이 최고에요…\n승자님 맛난거 드시길 '}
{'tweet_message': "청담 텐파마라탕 \n\n여전히 마싯는 꿔바로우 (o^-')b !\n\n#마크한테만공개\n#런쥔이_밥무거쒀 #런쥔이의_맛집내비게이션\n#먹다가_해찬이_생각이_나신거예요\n#박지성_단1g도안줌"}
{'tweet_message': '강남 마르르\n\n마르르 먹었어요..ㅎㅎ 경수야 맛있더라.. \n압구정로데오 맛집이심\n서울숲 광야갔다가 가도 좋을듯!!\n\n#백현이를_위한_맛집투어'}
{'tweet_messag

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
num_samples = 25
examples = [
    
]
for sample in messages[:num_samples]:
    examples.append(
        {
            "input": sample["tweet_message"],
            "output": sample["brand_name"],
        }
    )

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

print(few_shot_prompt.format())


Human: 신사 골뱅이신사

골뱅이 존맛 배터지게 먹고왔어용
#정한아_마니머거여ㅎㅎ
#믿고먹어boo세요
AI: 골뱅이신사
Human: 더보이즈 투어 폼 미춋다..
예절샷 못 찍는 사람들의 만남
 벽돌해피푸드 압구정점
 디저트로이
 파스텔 드 나따 압구정점
 한스 압구정점
 카츠바이콘반

#ㄷㅂㅇㅈ
#NewFoodShot
#지창민_도토리대신
AI: 벽돌해피푸드 / 디저트로이 / 파스텔드나따 / 한스 / 카츠바이콘반
Human: 코엑스 스시한판

직원이 졸라 불친절해서 개빡쳤지만
초밥 개맛있어서 화가 녹아버림 
콩띠소녀 첫 개시한 포카와 함께ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ

#ㄷㅂㅇㅈ
#상상그이상의_맛집
#여보세요_상연아_나_맛집이야_끊어
#맛또리를_찾아서
#끼끼한끼
AI: 스시한판
Human: 민현이가 좋아하는 압구정 로데오 포케 맛집
보울룸 <Bowl Room>
이라고합니다 •• ㅇㅋ접수..
 #황제님을위한메뉴판
AI: 보울룸
Human: 진짜 맛있게 배터지게 먹고 엄청 웃어서 소화 다 됐다.. 나나투어 직전에 벌떡 일어나서 해산

압구정 뱃고동
#원우야_여기_테이스티 
AI: 뱃고동
Human: 신사 피치

여기는 딸기조각케이크를 꼭꼭 드셔보세요 제발요
시트가 엄청 촉촉하구 얼그레이 시트인듯 ..?
생크림도 질리지않는 맛있는 맛이엿음 ㅠㅠ
한판 다먹을수있어 ,,,,,

#ㄷㅂㅇㅈ
#김선우_맛있는걸내가놔둘리가
#지창민_도토리대신
AI: 피치
Human: 크리스탈제이드 압구정 본점

하 여기 진짜진짜 개마싯음.. 탕수육 개 쫜득하고
샤오롱바오가 개.미친놈 웨이팅 기다릴 만한 맛
그리고 탄탄면이 엄청 진해요오..... 

#ㄷㅂㅇㅈ
#김선우_맛있는걸내가놔둘리가
#지창민_도토리대신
AI: 크리스탈제이드
Human: 해목 논현점
장어덮밥도 맛있고 후식으로 모찌리도후 추천
#런쥔이_밥무거쒀
#먹다가_해찬이_생각이나신거에요
AI: 해목
Human: #우무이
#우즈야_무즈는_이거먹어
#승연아_우즈야_여기야
공감 신사점

얼큰차돌 순두부나베랑 깐풍 칠리새우 진짜 개존맛탱!

In [38]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You extracts brand name from the korean tweeter message. Follow the examples."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)



functions = [
    {
        "name": "extract_brand",
        "description": "Extract brand names from korean tweeter message.",
        "parameters": {
            "type": "object",
            "properties": {
                "brand_name": {"type": "string", "description": "The brand name in the message"},
                "brand_name_with_region": {
                    "type": "string",
                    "description": "The full brand name with regional information.",
                },
            },
            "required": ["brand_name"],
        },
    }
]
model = ChatOpenAI()
chain = (
    final_prompt
    | model.bind(function_call={"name": "extract_brand"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [46]:
batch_input = [{"input": m["tweet_message"]} for m in messages[num_samples: num_samples+10]]
res=chain.batch(batch_input, batch_size = 5)

[{'brand_name': '텐파마라탕'},
 {'brand_name': '마르르'},
 {'brand_name': '이치서울'},
 {'brand_name': '카이센동'},
 {'brand_name': '앤드밀'},
 {'brand_name': '삼다연'},
 {'brand_name': '앤드밀'},
 {'brand_name': '압구정로데오'},
 {'brand_name': '미엘'},
 {'brand_name': '앤드밀'}]

TypeError: BaseChatModel.__call__() missing 1 required positional argument: 'messages'